In [102]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100

import numpy as np
import matplotlib.pyplot as plt

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [104]:
data_all = CIFAR100(root='/content/', train=True, download=True, transform=transforms.ToTensor())
data_all[0]

Files already downloaded and verified


(tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.7647, 0.8314, 0.7137],
          [1.0000, 0.9961, 0.9961,  ..., 0.6667, 0.6314, 0.5725],
          [1.0000, 0.9961, 1.0000,  ..., 0.7412, 0.6510, 0.4745],
          ...,
          [0.5804, 0.5569, 0.5490,  ..., 0.1176, 0.2549, 0.2980],
          [0.4784, 0.4706, 0.4941,  ..., 0.0863, 0.3804, 0.5529],
          [0.3412, 0.3451, 0.3961,  ..., 0.1333, 0.4118, 0.5412]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 0.8039, 0.8784, 0.7608],
          [1.0000, 0.9961, 0.9961,  ..., 0.6902, 0.6588, 0.6039],
          [1.0000, 0.9961, 1.0000,  ..., 0.7804, 0.6980, 0.5216],
          ...,
          [0.7255, 0.7137, 0.7020,  ..., 0.0667, 0.2431, 0.3020],
          [0.6157, 0.6078, 0.6275,  ..., 0.0627, 0.4392, 0.6314],
          [0.4784, 0.4784, 0.5255,  ..., 0.1412, 0.5216, 0.6784]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 0.7569, 0.8000, 0.6549],
          [1.0000, 0.9961, 0.9961,  ..., 0.5882, 0.5098, 0.4431],
          [1.0000, 0.9961, 1.0000,  ...,

In [105]:
RGB_mean = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in data_all]
RGB_std = [np.std(x.numpy(), axis=(1, 2)) for x,_ in data_all]

R_mean = np.mean([np.mean(m[0]) for m in RGB_mean])
G_mean = np.mean([np.mean(m[1]) for m in RGB_mean])
B_mean = np.mean([np.mean(m[2]) for m in RGB_mean])

R_std = np.mean([np.mean(n[0]) for n in RGB_std])
G_std = np.mean([np.mean(n[1]) for n in RGB_std])
B_std = np.mean([np.mean(n[2]) for n in RGB_std])

print(R_mean, G_mean, B_mean)
print(R_std, G_std, B_std)

0.50707513 0.48654887 0.44091785
0.20079845 0.19834627 0.20219834


In [106]:
data_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize(256),
                                      transforms.RandomResizedCrop(227),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.Normalize(mean=(R_mean, G_mean, B_mean), std =(R_std, G_std, B_std)),
                                      ])

train_ratio = 0.7
train_data_len = int(len(data_all) * train_ratio)
val_data_len = int(len(data_all)) - train_data_len

print(len(data_all))
print(train_data_len, val_data_len)

data_all = CIFAR100(root='/content/', train=True, download=True, transform=data_transforms)
train_data, val_data = random_split(data_all, [train_data_len, val_data_len])
test_data = CIFAR100(root='/content/', train=False, download=True, transform=data_transforms)


train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=128, shuffle=True, num_workers=2)

50000
35000 15000
Files already downloaded and verified
Files already downloaded and verified


In [107]:
a, b = next(iter(train_loader))
a.shape, b.shape

(torch.Size([128, 3, 227, 227]), torch.Size([128]))

In [108]:
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.ConvNet = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    self.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(256*6*6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 1000),
        # 아래 nn.Linear(1000, 100)는 데이터 클래스 수에 맞추기 위해 추가로 넣은 부분
        nn.Linear(1000, 100),
    )

    self.init_weight()

  def init_weight(self):
    for layer in self.ConvNet:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean=0, std=0.01)
        nn.init.constant_(layer.bias, 0)

      nn.init.constant_(self.ConvNet[4].bias, 1)
      nn.init.constant_(self.ConvNet[10].bias, 1)
      nn.init.constant_(self.ConvNet[12].bias, 1)

  def forward(self, x):
    x = self.ConvNet(x)
    x = x.view(-1, 256*6*6)
    x = self.fc(x)
    return x

In [109]:
import torchsummary

model = AlexNet()
model.to(device)

# AlexNet의 Image 입력 사이즈는 (3, 227, 227) 입니다.
torchsummary.summary(model, input_size=(3, 227, 227), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
 LocalResponseNorm-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
 LocalResponseNorm-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

In [110]:
for p in model.parameters():
  print(p)
  break

Parameter containing:
tensor([[[[-1.1187e-02,  3.4087e-03, -5.5355e-03,  ..., -1.7762e-04,
           -1.9762e-02,  5.4772e-03],
          [ 6.1632e-03, -2.8996e-03, -1.0003e-02,  ..., -6.4483e-03,
           -7.9030e-03, -4.5797e-03],
          [-1.1392e-02,  1.0070e-02, -1.1245e-02,  ..., -8.1451e-03,
            2.6818e-03,  4.8964e-03],
          ...,
          [-1.2312e-02, -1.5638e-03,  1.0077e-02,  ...,  1.0749e-02,
           -1.0649e-02, -1.5297e-02],
          [-7.5961e-03,  3.7821e-03,  1.0283e-02,  ..., -1.1246e-02,
            5.8392e-03,  9.1465e-03],
          [ 1.1848e-02,  4.7526e-03, -1.7935e-02,  ...,  4.7729e-03,
           -2.0722e-03, -8.0336e-03]],

         [[-2.1020e-02,  4.7299e-03, -4.0422e-03,  ...,  1.1225e-02,
            1.1703e-03, -4.9865e-03],
          [ 1.0281e-02, -7.8909e-03, -6.3852e-04,  ...,  6.1462e-03,
            1.7508e-02,  8.6734e-03],
          [ 7.4741e-03,  1.4452e-02,  6.1263e-03,  ...,  1.2738e-02,
            3.5315e-03,  8.9081e-03]

In [111]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(opt, step_size=50, gamma=0.1)

연구에서 제안한 모델에 학습 시킨 데이터셋의 클래스 자체가 1000개에 맞춰져있고,

용량 문제로 CIFAR100의 데이터로 학습을 진행하다 보니 원활한 학습이 불가능

전역한 후에는 논문에서 사용한 데이터를 사용하여 제대로 학습을 진행해보도록 하겠다.

In [113]:
for epoch in range(1):

  total_steps = 1

  for step, (X,y) in enumerate(train_loader, 0):
    X, y = X.to(device), y.to(device)
    opt.zero_grad()
    outputs = model(X)
    loss = loss_func(outputs, y).to(device)
    loss.backward()
    opt.step()

    if total_steps % 10 == 0:
      print(f'step: {total_steps} | Loss: {loss}')

    total_steps += 1
  lr_scheduler.step()

step: 10 | Loss: 4.603474140167236
step: 20 | Loss: 4.6061201095581055
step: 30 | Loss: 4.606268405914307
step: 40 | Loss: 4.607051372528076
step: 50 | Loss: 4.606911659240723
step: 60 | Loss: 4.60936975479126
step: 70 | Loss: 4.606873989105225
step: 80 | Loss: 4.605297565460205
step: 90 | Loss: 4.6112542152404785
step: 100 | Loss: 4.601169109344482
step: 110 | Loss: 4.608487606048584
step: 120 | Loss: 4.605777740478516
step: 130 | Loss: 4.609609127044678
step: 140 | Loss: 4.607582092285156
step: 150 | Loss: 4.605055809020996
step: 160 | Loss: 4.606514930725098
step: 170 | Loss: 4.6058454513549805
step: 180 | Loss: 4.603798866271973
step: 190 | Loss: 4.607887268066406
step: 200 | Loss: 4.605286121368408
step: 210 | Loss: 4.603733062744141
step: 220 | Loss: 4.60666036605835
step: 230 | Loss: 4.605386734008789
step: 240 | Loss: 4.607427597045898
step: 250 | Loss: 4.603980541229248
step: 260 | Loss: 4.6068620681762695
step: 270 | Loss: 4.608003616333008
